In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict

In [4]:
example_submit = pd.read_csv('data/HeadHunter_sample_submit.csv')
train = pd.read_csv('data/HeadHunter_train.csv')
test = pd.read_csv('data/HeadHunter_test.csv')

In [7]:
import nltk
import pymorphy2

nltk.download('punkt')
# probability score threshold
prob_thresh = 0.9
morph = pymorphy2.MorphAnalyzer()

def has_name(text):
    text = str(text)
    for word in nltk.word_tokenize(text):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh:
                return 1
    return 0

[nltk_data] Downloading package punkt to /home/ivan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
train['pos_has_name'] = train.positive.apply(has_name)
train['neg_has_name'] = train.negative.apply(has_name)

test['pos_has_name'] = test.positive.apply(has_name)
test['neg_has_name'] = test.negative.apply(has_name)

In [12]:
train.sample(5)

,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,rest_recovery_rating,target,pos_has_name,neg_has_name
35574,135370,Москва,Специалист по работе с клиентами,Надёжность. Честность.,Не знаю. Обучение. Больше ролевок.,4,5,4,3,5,4,0,0,0
24232,91943,Киев,Оператор,"Начинал свою карьеру на заводе с грузчика, пот...",нет,5,5,5,5,5,5,0,0,0
50057,189770,Хабаровск,"Старший инструктор, педагог","Близость к работе, система отпусков",Техническое обеспечение рабочего места,5,5,5,5,5,5,8,0,0
17096,64534,Москва,Работник зала,"берут абсолютно всех, без разбора, кто получше...",никому не советую побывать в этой эмоционально...,1,3,1,2,2,2,8,0,0
30269,115317,Екатеринбург,Менеджер по продажам оборудования,"Дмс и всякие плюшки, зп вся белая. Карьерный р...",Поднять зарплаты. Компания большая и деньги ес...,2,5,2,2,5,3,8,0,0


In [14]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import re

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    text = str(text)
    text = re.sub('[0-9]+', 'num', text)
    text = text.replace('******', 'компания')
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

[nltk_data] Downloading package stopwords to /home/ivan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /home/ivan/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [15]:
train.positive = train.positive.apply(preprocess_text)
train.negative = train.negative.apply(preprocess_text)

test.positive = test.positive.apply(preprocess_text)
test.negative = test.negative.apply(preprocess_text)

train.position = train.position.apply(preprocess_text)
test.position = test.position.apply(preprocess_text)

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker(distance=1, language='ru')

def correct_spelling(word):
    return spell.correction(word)

train.positive = train.positive.apply(correct_spelling)
train.negative = train.negative.apply(correct_spelling)

test.positive = test.positive.apply(correct_spelling)
test.negative = test.negative.apply(correct_spelling)

train.position = train.position.apply(correct_spelling)
test.position = test.position.apply(correct_spelling)

In [22]:
add_cities = {'могилев-подольский': 30589, 
              'днепр': 966400,
              'мелитополь': 154839,
              'хмельницкий':265693,
              'северская станица':24867,
              'лисаковск':36011,
              'курахово':18782,
              'семей':320397 ,
              'усть-каменогорск':326498,
              'тараз':358806,
              'нур-султан':1228800,
              'одесса':993120,
              'кременчуг':219022,
              'петергоф':84930,
              'кызылорда':242462,
              'луцк':213950,
              'матвеев-курган':41218,
              'староконстантинов':34455,
              'динская станица':34848,
              'бердянск':113354,
              'брест':343985,
              'турция':-1,
              'ленинградская станица':36940,
              'кривой рог':634780,
              'бованенково':-1,
              'краснодон':42774,
              'винница':370834,
              'алматы':1977011,
              'кировоград':226491 ,
              'костанай':237478,
              'риддер':48008,
              'северодонецк':103479,
              'актобе':413918,
              'андижан':416243,
              'краматорск':157175,
              'барановичи':179439,
              'тернополь':216384,
              'моршин':5754,
              'адлер':76534,
              'бердичев':74839,
              'гомель':508839,
              'житомир':266106,
              'туркменистан':-1,
              'харьков':1443886,
              'белая церковь':203816,
              'жезказган':87200,
              'колумбия':-1,
              'армения':-1,
              'зеленоград':250173,
              'шишкин лес':3913,
              'таиланд':-1,
              'багамские острова':-1,
              'мостиска':9411,
              'сестрорецк':43060,
              'бишкек':976734,
              'скадовск':17640,
              'бобруйск':217940,
              'чернигов':285821,
              'павлоград':108685,
              'киев':2954564,
              'сарны':29066,
              'овруч':15551,
              'астара':16130,
              'агсу':20100,
              'бровары':108349,
              'гродно':368710,
              'атырау':235314,
              'измаил':73500,
              'бангладеш':-1,
              'светловодск':44466,
              'бородянка':12535,
              'ужгород':112447,
              'кокшетау':163008,
              'черновцы':262276,
              'великобритания':-1,
              'полоцк':80795,
              'таджикистан':-1,
              'чехия':-1,
              'могилев':357100,
              'талдыкорган':144504,
              'солёное':7487,
              'кронштадт':44353,
              'славянск':111084,
              'смела':66972,
              'баку':2236000,
              'шымкент':932235,
              'зайсан':14389,
              'сабетта':22000,
              'кандыагаш':29169,
              'брюховецкая станица':22139,
              'лида':101165 }

features_to_sum = ['population', 'children']
for feature in features_to_sum:
    cities = pd.read_csv("data/cities.csv")
    cities = cities[['settlement', feature]]
    cities = cities.groupby('settlement').sum()
    cities.reset_index(level=0, inplace=True)
    cities_to_feature = dict(zip(cities.settlement, cities[feature]))
    f = lambda e: cities_to_feature[str(e).strip()] if str(e).strip() in cities_to_feature else -1
    train[feature] = train.city.apply(f)
    test[feature] = test.city.apply(f)
    
    if feature == 'population':
        add_cities_f = lambda e: add_cities[str(e).strip().lower()] if f(e) == -1 and str(e).strip().lower() in add_cities else f(e)
        train['population'] = train.city.apply(add_cities_f)
        test['population'] = test.city.apply(add_cities_f)

In [24]:
features_to_avg = ['latitude_dd', 'longitude_dd']
for feature in features_to_avg:
    cities = pd.read_csv("data/cities.csv")
    cities = cities[['settlement', feature]]
    cities = cities.groupby('settlement').mean()
    cities.reset_index(level=0, inplace=True)
    cities_to_feature = dict(zip(cities.settlement, cities[feature]))
    f = lambda e: cities_to_feature[str(e).strip()] if str(e).strip() in cities_to_feature else -1
    train[feature] = train.city.apply(f)
    test[feature] = test.city.apply(f)

In [25]:
train["positive_len"] = train.positive.apply(len)
train["negative_len"] = train.negative.apply(len)
train["position_len"] = train.position.apply(len)

In [26]:
test["positive_len"] = test.positive.apply(len)
test["negative_len"] = test.negative.apply(len)
test["position_len"] = test.position.apply(len)

In [27]:
from collections import Counter

k = 50
top_k_words_for_cls = dict()
for label in range(9):
    idxs = [str(label) in s for s in train['target']]
    counter = Counter(' '.join(train[idxs].position).split())
    top_k_words_for_cls[label] = list(sorted(counter.items(), key=lambda e: -e[1]))[:k]

In [28]:
label = 2
idxs = [str(label) in s for s in train['target']]
train[idxs].positive

7463                               удаленный формат работа
10174    прочитывать отзыв сотрудник компания работодат...
13299                 возможность работать дом офис предел
14264    заинтересовывать вакансия компания немного сму...
19978                        стабильность доверие компания
21763    work life balance возможность работать удаленн...
22731    работать мерчандайзер num num год недавно увол...
27201    добрый время сутки хотеться обращаться лично н...
27351    наверно жизнь отставать понимать почему недово...
30570                                 свобода выбор график
49725    собеседование проходить легко хотя волноваться...
50154    работать компания num год очень нравиться пони...
50396    оплачиваться весь переработка праздничный день...
Name: positive, dtype: object

In [29]:
train[idxs].negative

7463     коммуникация отдел кадры непонятный ситуация т...
10174    хотеться лишний выходной ... это наверное везд...
13299                               коммуникация сотрудник
14264    пока устраивать далеко видно мочь находиться н...
19978    увеличивать количество персонал магазин очень ...
21763    возобновлять совместный выездной встреча коман...
22731    прорабатывать num num год находить минус хотя ...
27201    многое устраивать сравнивать ... достойный орг...
27351    далеко ехать работа это суть проблема многий р...
30570                   зарплата нагрузка рабочий персонал
49725    отзыв сотрудник компания часто читать ладиться...
50154    корпоративный мероприятие итак хотеться провож...
50396    одноименный группа вк подслушивать компания от...
Name: negative, dtype: object

In [30]:
train[idxs].position

7463             кредитный инспектор
10174                            nan
13299         специалист страхование
14264                            nan
19978    территориальный управляющий
21763                       рекрутер
22731                   мерчандайзер
27201            консультант эксперт
27351                            nan
30570       специалист контакт центр
49725                            nan
50154                       провизор
50396             специалист продажа
Name: position, dtype: object

In [31]:
terms = ['gmail', 'commarystarkova', 'ссылка', 'адрес']
for term in terms:
    train["positive_contains_" + term] = train.positive.apply(lambda e: int(term in e))
    train["negative_contains_" + term] = train.negative.apply(lambda e: int(term in e))
    
    test["positive_contains_" + term] = test.positive.apply(lambda e: int(term in e))
    test["negative_contains_" + term] = test.negative.apply(lambda e: int(term in e))    

In [51]:
from tqdm import tqdm
train_leven = list()
for pair in tqdm(zip(train.positive, train.negative)):
    train_leven.append(nltk.edit_distance(pair[0], pair[1]))

50876it [21:11, 40.01it/s] 


In [52]:
test_leven = list()
for pair in tqdm(zip(test.positive, test.negative)):
    test_leven.append(nltk.edit_distance(pair[0], pair[1]))

31245it [11:17, 47.85it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

50651it [17:20, 48.70it/s]


In [53]:
train["edit_d"] = train_leven
test["edit_d"] = test_leven

In [ ]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [ ]:
messages = train.negative.tolist()
results = model.predict(messages, k=5)
for feature in ['neutral', 'skip', 'negative', 'positive', 'speech']:
    l = list()
    for result in results:
        l.append(result[feature])
    train['negative_dost:' + feature] = l

In [ ]:
messages = train.positive.tolist()
results = model.predict(messages, k=5)
for feature in ['neutral', 'skip', 'negative', 'positive', 'speech']:
    l = list()
    for result in results:
        l.append(result[feature])
    train['positive_dost:' + feature] = l

In [ ]:
messages = test.negative.tolist()
results = model.predict(messages, k=5)
for feature in ['neutral', 'skip', 'negative', 'positive', 'speech']:
    l = list()
    for result in results:
        l.append(result[feature])
    test['negative_dost:' + feature] = l

In [ ]:
messages = test.positive.tolist()
results = model.predict(messages, k=5)
for feature in ['neutral', 'skip', 'negative', 'positive', 'speech']:
    l = list()
    for result in results:
        l.append(result[feature])
    test['positive_dost:' + feature] = l

In [ ]:
train.to_csv('data/HeadHunter_train_prep.csv', index=False)
test.to_csv('data/HeadHunter_test_prep.csv', index=False)